# Party Outcome Predictions & Group Fairness
 
This tutorial examine a docket's dataset from the Federal Judical Center.  We'll focus on Torts from 2010 to 2019.  The objective is to predict if the case's outcome is in the favor of the Plaintiff.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 


df = pd.read_csv('/kaggle/input/iltacon-fair-ai-tutorial/preped_dataset.csv')

# Install Fair AI packages

1. [Surgio](https://surgeo.readthedocs.io/en/dev/) - Python package to deterime race by proxy variables (i.e. given name, surname, geocode)
2. [AI Fairness 360](https://aif360.mybluemix.net/) - An API to measure fairness and mitigate bias in machine learning datasets and models.

In [ ]:
pip install -U -q ipympl /kaggle/input/iltacon-fair-ai-tutorial/surgeo-1.2.0-py3-none-any.whl 'aif360[all]'

# Create Labels
Was the outcome in favor of the Plaintiff?

For the demo, we'll mark
* Judgment in favor of Plaintiff (or Both parties)
* Disposition is a settlement 
* If the judgment is unknown, then default and consent judgments are considered favorable.

> Note: We're only determining if the plaintiff receives a judgment/settlement and not if the judgment was equitable  

In [ ]:
def for_plaintiff(row):
    judgment = row['JUDGMENT']
    disp = row['DISP']
    if judgment in [1,3] or disp == 13 or (judgment != 2 and disp in [4,5]):
        return 1.0
    else:
        return 0.0

df['for_plaintiff'] = df.apply(for_plaintiff, axis=1)
print(df['for_plaintiff'].value_counts())

labeled_df = df.drop(columns=['NOJ','JUDGMENT','DISP','DEF']).dropna().reset_index()

sns.countplot(labeled_df['for_plaintiff'])
plt.title("favors plaintiff",color = 'blue',fontsize=18)

# Label Privileged Classes

Although the federal docket dataset doesn't contain explicit race or gender columns, the dataset includes some income attributes.  These include:
1. Informa Pauperis (uanble to pay court costs)
2. Pro Se (not represented by a laywer)

In [ ]:
sns.countplot(labeled_df['is_ifp'])
plt.title("Informa Pauperis (court fees waived)",color = 'blue',fontsize=18)

In [ ]:
sns.countplot(labeled_df['pro_se_plt'])
plt.title("Pro Se plaintiff (i.e. no lawyer)",color = 'blue',fontsize=18)

In [ ]:
sns.countplot(labeled_df['pro_se_def'])
plt.title("Pro Se defendant (i.e. no lawyer)",color = 'blue',fontsize=18)

# Dataset 

Using the docket's metadata predict if the case's outcome was in the favor of the Plaintiff.

From the Panda's dataframe:
* Select to column's label
* List the protected attributes
* List categorical features (i.e. the district the case was filed in)
* List features to keep (i.e. the number of days the case was opened)


In [ ]:
from aif360.datasets import StandardDataset
categorial_features = ['NOS','plaintiff_res','def_res','JURIS','PROCPROG']
features_to_keep = ['NOS',
                    'plaintiff_res',
                    'def_res',
                    'JURIS',
                    'PROCPROG',
                    'county_pop',
                    'filing_rate_mean',
                    'filing_rate_stdev',
                    'dismiss_rate_mean',
                    'dismiss_rate_stdev',
                    'no_demand',
                    'demanded_imp',
                    'days_open',
                    'is_joined',
                    'trial_began']

ds = StandardDataset(labeled_df, 
                     label_name='for_plaintiff', 
                     favorable_classes=[1.0],
                     protected_attribute_names=['is_ifp'], 
                     privileged_classes=[[0.0]],
                     categorical_features=categorial_features,
                     features_to_keep=features_to_keep)


# Fairness Metrics 

__Consistency:__
    
Individual fairness metric that measures how similar the labels are for similar instances (0.0  to 1.0).
    
$1 - \frac{1}{n}\sum_{i=1}^n |\hat{y}_i -
\frac{1}{\text{n_neighbors}} \sum_{j\in\mathcal{N}_{\text{n_neighbors}}(x_i)} \hat{y}_j|$

__Disparate Impact:__   

Probability postive outcomes of unprivileged group over privilaged group (0 to $\infty$).  If we were looking at employment by sex, race, or ethinic group, then a disparate impact less than 80% is considered discriminatory (29 CFR § 1607.4).
    
$\frac{Pr(\hat{Y} = \text{pos_label} | D = \text{unprivileged})}{Pr(\hat{Y} = \text{pos_label} | D = \text{privileged})}$

__Statistical Parity Difference:__

Probability postive outcomes difference between unprivileged and privilaged groups (1 to -1).
    
$Pr(\hat{Y} = \text{pos_label} | D = \text{unprivileged}) - Pr(\hat{Y} = \text{pos_label} | D = \text{privileged})$

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric
priv_group = [{'is_ifp': 0.0}]
unpriv_group = [{'is_ifp': 1.0}]
ifp_metrics = BinaryLabelDatasetMetric(ds, unpriv_group, priv_group)
consistency = round(ifp_metrics.consistency()[0], 4)
disparate_impact = round(ifp_metrics.disparate_impact(), 4)
statistical_parity_difference = round(ifp_metrics.statistical_parity_difference(), 4)
print(f'Consistency: {consistency}')
print(f'Disparate Impact: {disparate_impact}')
print(f'Statistical Parity Difference: {statistical_parity_difference}')

In [ ]:
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from aif360.algorithms.preprocessing import DisparateImpactRemover

In [ ]:
protected = 'is_ifp'
test = ds.subset(labeled_df[labeled_df['TAPEYEAR'] == 2015].index)
train = ds.subset(labeled_df[labeled_df['TAPEYEAR'] < 2015].index)

scaler = MinMaxScaler(copy=False)
train.features = scaler.fit_transform(train.features)
test.features = scaler.fit_transform(test.features)

index = train.feature_names.index(protected)

In [ ]:
DIs = []
for level in tqdm(np.linspace(0., 1., 11)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(train)
    test_repd = di.fit_transform(test)
    
    X_tr = np.delete(train_repd.features, index, axis=1)
    X_te = np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    lmod = LogisticRegression(class_weight='balanced', solver='liblinear')
    lmod.fit(X_tr, y_tr)
    
    test_repd_pred = test_repd.copy()
    test_repd_pred.labels = lmod.predict(X_te)

    p = [{protected: 0.0}]
    u = [{protected: 1.0}]
    cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
    DIs.append(cm.disparate_impact())

In [ ]:
%matplotlib inline

plt.plot(np.linspace(0, 1, 11), DIs, marker='o')
plt.plot([0, 1], [1, 1], 'g')
plt.plot([0, 1], [0.8, 0.8], 'r')
plt.ylim([0.0, 1.3])
plt.ylabel('Disparate Impact (DI)')
plt.xlabel('repair level')
plt.show()

# Estimating Race

The Dockets don’t contain race but may contain fields that are proxies for race.  In order to see the imapct of proxies on an machine learning algorithm, we're going to estimate the plaintiff's race.  We'll use [Bayesian Improved Surname Geocode (BISG)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1797082/) to estimate race based on US Cenus Data.  

In [ ]:
import re

def last_name(party: str):
    """Extract lastname form party."""
    tokens = re.split(r'[.,\s]+', party)
    if(len(tokens) == 1):
        #Assume is party is a person if only one word
        return tokens[0]
    elif len(tokens) == 3 and ' '.join(tokens).endswith(' ET AL'):
        #Assume first name listed is the primary person
        return tokens[0]
    else:
        #Assume longer names are orginzations
        return None

plt_df = df[df['PLT'].notnull()].copy()
plt_df['PLT_LASTNAME'] = plt_df['PLT'].apply(last_name)
person_df = plt_df[(plt_df['PLT_LASTNAME'].notnull()) & (plt_df['COUNTY'].notnull())].copy().reset_index()

In [ ]:
person_df[['PLT','PLT_LASTNAME','COUNTY','county_name']].head(10)

## Surgeo 

[Surgeo](https://github.com/theonaunheim/surgeo) is a python package to estimate race base on zip codes, surname, and forename.

> Note: We've modified surgeo to use county codes instead of zip codes.

In [ ]:
import surgeo

# Instatiate your model
surgeo_model = surgeo.SurgeoModel(geo_level="FIPSCC")
surgeo_df = surgeo_model.get_probabilities(person_df['PLT_LASTNAME'], person_df['COUNTY'])

def race_label(row):
    prob_threshold = 0.5
    for race in ['black','api','native','hispanic','white']:
        if row[race] >= prob_threshold:
            return race    
    
    return 'uncertain'

surgeo_df['race'] = surgeo_df.apply(race_label, axis=1)
        
surgeo_df.sample(n=20, random_state=1)

In [ ]:
sns.countplot(surgeo_df['race'])
plt.title("Primary Plaintiff by Race (threshold > 0.5)",color = 'blue',fontsize=18)

In [ ]:
prob_df = pd.concat([person_df,surgeo_df['white']], axis=1)
proxy_df = prob_df.copy().dropna().reset_index()
proxy_df['white'] = proxy_df['white'].apply(lambda x: 1.0 if x > 0.5 else 0.0)
print(proxy_df['white'].value_counts())

sns.countplot(proxy_df['white'])

In [ ]:
from aif360.datasets import StandardDataset
proxy_ds = StandardDataset(proxy_df, 
                     label_name='for_plaintiff', 
                     favorable_classes=[1.0],
                     protected_attribute_names=['white'], 
                     privileged_classes=[[1.0]],
                     categorical_features=categorial_features,
                     features_to_keep=features_to_keep)

## Ground Truth Metrics (entire dataset)

In [ ]:
from aif360.metrics import BinaryLabelDatasetMetric
priv_group = [{'white': 1.0}]
unpriv_group = [{'white': 0.0}]
race_metrics = BinaryLabelDatasetMetric(proxy_ds, unpriv_group, priv_group)
consistency = round(race_metrics.consistency()[0],4)
disparate_impact = round(race_metrics.disparate_impact(),4)
statistical_parity_difference = round(race_metrics.statistical_parity_difference(),4)

print(f'Consistency: {consistency}')
print(f'Disparate Impact: {disparate_impact}')
print(f'Statistical Parity Difference: {statistical_parity_difference}')

# Plaintiff Outcome Classifier

We're going to compare fairness metrics between the ground truth labels and a classifier.

In [ ]:
protected = 'white'
p = [{protected: 1.0}]
u = [{protected: 0.0}]
test = proxy_ds.subset(proxy_df[proxy_df['TAPEYEAR'] == 2015].index)
train = proxy_ds.subset(proxy_df[proxy_df['TAPEYEAR'] < 2015].index)

scaler = MinMaxScaler(copy=False)
train.features = scaler.fit_transform(train.features)
test.features = scaler.fit_transform(test.features)

index = train.feature_names.index(protected)

### 2015 Ground Truth Dataset Metrics

__Disparate Impact:__   

Probability postive outcomes of unprivileged group over privilaged group (0 to $\infty$).
    
$\frac{Pr(\hat{Y} = \text{pos_label} | D = \text{unprivileged})}
{Pr(\hat{Y} = \text{pos_label} | D = \text{privileged})}$

__Smoothed Empirical Differential Fairness:__

J. R. Foulds, R. Islam, K. N. Keya, and S. Pan, “An Intersectional Definition of Fairness,” arXiv preprint arXiv:1807.08362, 2018.

In [ ]:
ground_truth_ds = BinaryLabelDatasetMetric(test, privileged_groups=p, unprivileged_groups=u)
disparate_impact = round(ground_truth_ds.disparate_impact(), 4)
smdf = round(ground_truth_ds.smoothed_empirical_differential_fairness(), 4)
print(f'Disparate Impact: {disparate_impact}')
print(f'Smoothed Empirical Differential Fairness: {smdf}')

### 2015 Predicted Dataset Fairness Metric

__Disparate Impact:__   
    
$\frac{Pr(\hat{Y} = \text{pos_label} | D = \text{unprivileged})}
{Pr(\hat{Y} = \text{pos_label} | D = \text{privileged})}$

__Smoothed Empirical Differential Fairness:__

J. R. Foulds, R. Islam, K. N. Keya, and S. Pan, “An Intersectional Definition of Fairness,” arXiv preprint arXiv:1807.08362, 2018.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_tr = np.delete(train.features, index, axis=1)
X_te = np.delete(test.features, index, axis=1)
y_tr = train.labels.ravel()

lmod = LogisticRegression(class_weight='balanced', solver='liblinear')
lmod.fit(X_tr, y_tr)
    
test_pred = test.copy()
test_pred.labels = lmod.predict(X_te)
classifier_ds = BinaryLabelDatasetMetric(test_pred, privileged_groups=p, unprivileged_groups=u)
cl_disparate_impact = round(classifier_ds.disparate_impact(), 4)
cl_smdf = round(classifier_ds.smoothed_empirical_differential_fairness(), 4)

print(f'Disparate Impact: {cl_disparate_impact}')
print(f'Smoothed Empirical Differential Fairness: {cl_smdf}')

### Classification Fairness

Lets look at the different between the ground truth dataset and the predicted dataset.  Is there any bias amplification?

__Average Odds Difference:__

Computed as average difference of false positive rate (false positives / negatives) and true positive rate (true positives / positives) between unprivileged and privileged groups.  The ideal value of this metric is 0.  A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group.

$\dfrac{(FPR_{D = \text{unprivileged}} - FPR_{D = \text{privileged}}) + (TPR_{D = \text{unprivileged}} - TPR_{D = \text{privileged}})}{2}$

__Equal Opportunity Difference:__

The true positive rate difference (TPR).  The ideal value is 0. A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group.

$TPR_{D = \text{unprivileged}} - TPR_{D = \text{privileged}}$


__Differential Fairness Bias Amplification:__

The extent to which the classifier increases the unfairness over the original data. 

J. R. Foulds, R. Islam, K. N. Keya, and S. Pan, “An Intersectional Definition of Fairness,” arXiv preprint arXiv:1807.08362, 2018.

In [ ]:
from aif360.metrics import ClassificationMetric
cm = ClassificationMetric(test, test_pred, privileged_groups=p, unprivileged_groups=u)
aod = round(cm.average_odds_difference(), 4)
eod = round(cm.equal_opportunity_difference(), 4)
bias_amplification = round(cm.differential_fairness_bias_amplification(), 4)

print(f'Average Odds Difference: {aod}')
print(f'Equal Oppertunity Difference: {eod}')
print(f'Differential Fairness Bias Amplification: {bias_amplification}')